In [1]:
import pandas as pd
import numpy as np
import datetime as dt

import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *#avg, count, expr
from pyspark.sql.types import *
from pyspark.ml.feature import VectorAssembler, OneHotEncoder, MinMaxScaler, StringIndexer
from pyspark.ml.regression import *
from pyspark.ml.evaluation import *
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.stat import Correlation
from pyspark.ml import Pipeline

In [2]:
# initialize
sc = pyspark.SparkContext()
spark = SparkSession(sc)
spark.sparkContext.appName = 'regressionHW'
# show the number of cores
print('%d cores'%spark._jsc.sc().getExecutorMemoryStatus().keySet().size())
spark

1 cores


In [4]:
''' get the data '''
# load the data
fil = '../../data/Concrete_Data.csv'
schem = StructType([StructField('cement', FloatType()), StructField('slag', FloatType()),
                    StructField('flyash', FloatType()), StructField('water', FloatType()),
                    StructField('superplasticizer', FloatType()), StructField('coarseaggregate', FloatType()),
                    StructField('fineaggregate', FloatType()), StructField('age', FloatType()),
                    StructField('csMPa', FloatType())])
concrete = spark.read.format('csv').options(header=True).schema(schem).load(fil)

# add an ID - don't actually care if it's monotonic
concrete = concrete.select(monotonically_increasing_id().alias('id'), '*')

# talk
cnt = concrete.count()
print('%d records'%cnt)
concrete.show(truncate=False)

1030 records
+---+------+-----+------+-----+----------------+---------------+-------------+-----+-----+
|id |cement|slag |flyash|water|superplasticizer|coarseaggregate|fineaggregate|age  |csMPa|
+---+------+-----+------+-----+----------------+---------------+-------------+-----+-----+
|0  |540.0 |0.0  |0.0   |162.0|2.5             |1040.0         |676.0        |28.0 |79.99|
|1  |540.0 |0.0  |0.0   |162.0|2.5             |1055.0         |676.0        |28.0 |61.89|
|2  |332.5 |142.5|0.0   |228.0|0.0             |932.0          |594.0        |270.0|40.27|
|3  |332.5 |142.5|0.0   |228.0|0.0             |932.0          |594.0        |365.0|41.05|
|4  |198.6 |132.4|0.0   |192.0|0.0             |978.4          |825.5        |360.0|44.3 |
|5  |266.0 |114.0|0.0   |228.0|0.0             |932.0          |670.0        |90.0 |47.03|
|6  |380.0 |95.0 |0.0   |228.0|0.0             |932.0          |594.0        |365.0|43.7 |
|7  |380.0 |95.0 |0.0   |228.0|0.0             |932.0          |594.0        

### Data Prep

In [5]:
''' handle missing values '''
# check for missing values
nullCounts = {colm:concrete.select(colm).where(col(colm).isNull()).count() for colm in concrete.columns}
nullCounts = {colm:(ncnt, ncnt/cnt) for (colm, ncnt) in nullCounts.items()}
nullCountsDF = pd.DataFrame(nullCounts).T.reset_index(drop=False).sort_values(1, ascending=False)
nullCountsDF.columns = ['Column', 'Freq.', 'Rel. Freq.']
nullCountsDF = nullCountsDF.merge(pd.DataFrame([[colm.name, colm.dataType] for colm in concrete.schema], columns=['Column', 'Type']),
                                how='inner', on=['Column'])

# talk
display(nullCountsDF)

# remove
#concrete = concrete.dropna(how='any')

# talk some more
print('%d records'%concrete.count())

,Column,Freq.,Rel. Freq.,Type
0,id,0.0,0.0,LongType
1,cement,0.0,0.0,FloatType
2,slag,0.0,0.0,FloatType
3,flyash,0.0,0.0,FloatType
4,water,0.0,0.0,FloatType
5,superplasticizer,0.0,0.0,FloatType
6,coarseaggregate,0.0,0.0,FloatType
7,fineaggregate,0.0,0.0,FloatType
8,age,0.0,0.0,FloatType
9,csMPa,0.0,0.0,FloatType


1030 records


In [6]:
# prepare the response
concrete = concrete.withColumnRenamed('csMPa', 'label')

In [7]:
''' prepare the features '''
# get the features
features = [c for c in concrete.columns if c not in (['label', 'id'])]

# create & scale the features vector
assr = VectorAssembler(inputCols=features, outputCol='features_raw')
scalr = MinMaxScaler(inputCol='features_raw', outputCol='features')
pipe = Pipeline(stages=[assr, scalr]).fit(concrete)
concrete = pipe.transform(concrete).drop('features_raw')

# talk
display(concrete.limit(10).toPandas())
concrete.select('id', 'features', 'label').show(truncate=False)
concrete.select('features').take(1)
print('First row features = %s'%concrete.select('features').take(1)[0])

,id,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,label,features
0,0,540.000000,0.000000,0.0,162.0,2.5,1040.000000,676.0,28.0,79.989998,"[1.0, 0.0, 0.0, 0.3210862454322655, 0.07763974..."
1,1,540.000000,0.000000,0.0,162.0,2.5,1055.000000,676.0,28.0,61.889999,"[1.0, 0.0, 0.0, 0.3210862454322655, 0.07763974..."
2,2,332.500000,142.500000,0.0,228.0,0.0,932.000000,594.0,270.0,40.270000,"[0.526255707762557, 0.39649416366168144, 0.0, ..."
3,3,332.500000,142.500000,0.0,228.0,0.0,932.000000,594.0,365.0,41.049999,"[0.526255707762557, 0.39649416366168144, 0.0, ..."
4,4,198.600006,132.399994,0.0,192.0,0.0,978.400024,825.5,360.0,44.299999,"[0.22054795914044661, 0.3683917533249004, 0.0,..."
5,5,266.000000,114.000000,0.0,228.0,0.0,932.000000,670.0,90.0,47.029999,"[0.3744292237442922, 0.31719533092934515, 0.0,..."
6,6,380.000000,95.000000,0.0,228.0,0.0,932.000000,594.0,365.0,43.700001,"[0.634703196347032, 0.264329442441121, 0.0, 0...."
7,7,380.000000,95.000000,0.0,228.0,0.0,932.000000,594.0,28.0,36.450001,"[0.634703196347032, 0.264329442441121, 0.0, 0...."
8,8,266.000000,114.000000,0.0,228.0,0.0,932.000000,670.0,28.0,45.849998,"[0.3744292237442922, 0.31719533092934515, 0.0,..."
9,9,475.000000,0.000000,0.0,228.0,0.0,932.000000,594.0,28.0,39.290001,"(0.8515981735159817, 0.0, 0.0, 0.8482428078025..."


+---+------------------------------------------------------------------------------------------------------------------------------+-----+
|id |features                                                                                                                      |label|
+---+------------------------------------------------------------------------------------------------------------------------------+-----+
|0  |[1.0,0.0,0.0,0.3210862454322655,0.07763974971321652,0.6947674418604651,0.2057200326705011,0.07417582417582418]                |79.99|
|1  |[1.0,0.0,0.0,0.3210862454322655,0.07763974971321652,0.7383720930232558,0.2057200326705011,0.07417582417582418]                |61.89|
|2  |[0.526255707762557,0.39649416366168144,0.0,0.8482428078025063,0.0,0.3808139534883721,0.0,0.739010989010989]                   |40.27|
|3  |[0.526255707762557,0.39649416366168144,0.0,0.8482428078025063,0.0,0.3808139534883721,0.0,1.0]                                 |41.05|
|4  |[0.22054795914044661,0

In [8]:
# check for multicollinearity
# high: 
corr = Correlation.corr(concrete, column='features', method='pearson')
corrdf = pd.DataFrame(index=features, data=corr.collect()[0][0].toArray(), columns=features)
display(corrdf)

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age
cement,1.000000,-0.275216,-0.397467,-0.081587,0.092386,-0.109349,-0.222718,0.081946
slag,-0.275216,1.000000,-0.323580,0.107252,0.043270,-0.283999,-0.281603,-0.044246
flyash,-0.397467,-0.323580,1.000000,-0.256984,0.377503,-0.009961,0.079109,-0.154371
water,-0.081587,0.107252,-0.256984,1.000000,-0.657533,-0.182294,-0.450661,0.277618
superplasticizer,0.092386,0.043270,0.377503,-0.657533,1.000000,-0.265999,0.222691,-0.192700
coarseaggregate,-0.109349,-0.283999,-0.009961,-0.182294,-0.265999,1.000000,-0.178481,-0.003016
fineaggregate,-0.222718,-0.281603,0.079109,-0.450661,0.222691,-0.178481,1.000000,-0.156095
age,0.081946,-0.044246,-0.154371,0.277618,-0.192700,-0.003016,-0.156095,1.000000


## Modeling

In [9]:
# split for cross-val
trainPerc = 0.7
randSeed = 42
tran, test = concrete.select('id', 'label', 'features').randomSplit([trainPerc, 1.0 - trainPerc], seed=randSeed)

# talk
print('Training Cases')
tran.select('id').show()
print('Testing Cases')
test.select('id').show()

Training Cases
+---+
| id|
+---+
|  0|
|  1|
|  3|
|  4|
|  5|
|  7|
| 10|
| 11|
| 12|
| 16|
| 17|
| 18|
| 20|
| 22|
| 25|
| 26|
| 27|
| 31|
| 33|
| 36|
+---+
only showing top 20 rows

Testing Cases
+---+
| id|
+---+
|  2|
|  6|
|  8|
|  9|
| 13|
| 14|
| 15|
| 19|
| 21|
| 23|
| 24|
| 28|
| 29|
| 30|
| 32|
| 34|
| 35|
| 39|
| 42|
| 43|
+---+
only showing top 20 rows



In [10]:
''' set up the estimators & param grids '''
models = {}

# linear regression
linreg = LinearRegression()
params = (ParamGridBuilder().addGrid(linreg.elasticNetParam, [0.0, 0.25, 0.5, 0.75, 1.0]).build())
paramNames = ['elasticnetparam']
models['linear regression'] = [linreg, params, paramNames, None, None]

# random forest
ranfor = RandomForestRegressor(numTrees=20)
params = (ParamGridBuilder().addGrid(ranfor.maxBins, [20, 40, 80, 100])\
              .addGrid(ranfor.maxDepth, [5, 10, 30]).build())
paramNames = ['maxbins', 'maxdepth']
models['random forest'] = [ranfor, params, paramNames, None, None]

# gradient boosting trees
gradbst = GBTRegressor(maxIter=20)
params = (ParamGridBuilder().addGrid(gradbst.maxBins, [20, 40, 80, 100])\
              .addGrid(gradbst.maxDepth, [5, 10, 30]).build())
paramNames = ['maxbins', 'maxdepth']
models['gradient boost'] = [gradbst, params, paramNames, None, None]

In [ ]:
''' run the models '''
# number of cv folds
folds = 5
# define the evaulation function
evl = RegressionEvaluator(metricName='rmse')

# iterate over models
for (model, stuff) in models.items():
    print('Cross Validator: %s'%model)
    # execute
    cv = CrossValidator(estimator=stuff[0], estimatorParamMaps=stuff[1], evaluator=evl, numFolds=folds)
    fitModel = cv.fit(concrete.select('features', 'label'))
    # get the best
    bestModel = fitModel.bestModel
    # evaluate performance on the test set
    testRMSE = evl.evaluate(bestModel.transform(test.select('features', 'label')))
    print('\tBest Model Test RMSE = %0.3f'%testRMSE)    
    # get best parameters
    bestParams = bestModel.extractParamMap()
    for (key, val) in bestParams.items():
        for parm in stuff[2]:
            if parm in key.name.lower():
                print('\t%s = %0.2f'%(key, val))
                break
    # save stuff
    models[model][3] = fitModel
    models[model][4] = testRMSE

Cross Validator: linear regression


In [ ]:
# look at linear regression coefficients
bm = models['linear regression'][3].bestModel
summ = bm.summary
summ.predictions.describe().withColumn('Diff', col('prediction') - col('label')).show(truncate=False)
print('Best Model RMSE = %0.3f'%summ.rootMeanSquaredError)

# get a nice model coefficients table
coefs = pd.concat([pd.DataFrame(index=['Intercept'], data=[bm.intercept], columns=['Coefficient']),
                   pd.DataFrame(index=features, data=bm.coefficients.toArray(), columns=['Coefficient'])])
coefs['Std. Error'] = bm.summary.coefficientStandardErrors
coefs['pValue'] = bm.summary.pValues
# make an absolute coef column temporarily for sorting
coefs['tmp'] = coefs['Coefficient'].abs() a
coefs.loc['Intercept', 'tmp'] = np.inf
coefs = coefs.sort_values(by='tmp', ascending=False).drop(columns='tmp')

# talk
display(coefs)

In [ ]:
# view feature importances for random forest
imports = models['random forest'][3].bestModel.featureImportances.toArray()
imports = pd.DataFrame(index=features, data=imports, columns=['Importance']).sort_values(by='Importance', ascending=False, inplace=False)
display(imports)

In [ ]:
# view feature importances for gradient boost
imports = models['gradient boost'][3].bestModel.featureImportances.toArray()
imports = pd.DataFrame(index=features, data=imports, columns=['Importance']).sort_values(by='Importance', ascending=False, inplace=False)
display(imports)

In [ ]:
sc.stop()